Checkpoint Objective
Logistic Regression:
In this checkpoint, we are going to work  on the heart disease dataset , this time we will use logistic regression to predict if a patient will have TenyearsCHD

1. Apply logistic regression.

2. Use a confusion matrix to validate your model.

3. Another validation matrix for classification is ROC / AUC. Do your research on them, explain them, and apply them in our case.


In [184]:
import pandas as pd
import plotly.graph_objects as go
# import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, auc
import matplotlib.pyplot as plt

heart_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/logistic_regression_dataset.csv")

In [185]:
# Task 2: Data Cleaning

missing_values = heart_data.isnull().sum()
print(missing_values)

id                   0
age                  0
education          105
sex                  0
is_smoking           0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD         848
dtype: int64


In [186]:
heart_data.head()

,id,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,0,64,2.0,F,YES,3.0,0.0,0,0,0,221.0,148.0,85.0,NaN,90.0,80.0,1.0
1,1,36,4.0,M,NO,0.0,0.0,0,1,0,212.0,168.0,98.0,29.77,72.0,75.0,0.0
2,2,46,1.0,F,YES,10.0,0.0,0,0,0,250.0,116.0,71.0,20.35,88.0,94.0,0.0
3,3,50,1.0,M,YES,20.0,0.0,0,1,0,233.0,158.0,88.0,28.26,68.0,94.0,1.0
4,4,64,1.0,F,YES,30.0,0.0,0,0,0,241.0,136.5,85.0,26.42,70.0,77.0,0.0


In [187]:
# Calculate the mean for each column
mean_education = heart_data['education'].mean()
mean_cigsPerDay = heart_data['cigsPerDay'].mean()
mean_BPMeds = heart_data['BPMeds'].mean()
mean_totChol = heart_data['totChol'].mean()
mean_BMI = heart_data['BMI'].mean()
mean_heartRate = heart_data['heartRate'].mean()
mean_glucose = heart_data['glucose'].mean()
mean_TenYearCHD = heart_data['TenYearCHD'].mean()

# Fill missing values with the mean for each column
heart_data['education'].fillna(mean_education, inplace=True)
heart_data['cigsPerDay'].fillna(mean_cigsPerDay, inplace=True)
heart_data['BPMeds'].fillna(mean_BPMeds, inplace=True)
heart_data['totChol'].fillna(mean_totChol, inplace=True)
heart_data['BMI'].fillna(mean_BMI, inplace=True)
heart_data['heartRate'].fillna(mean_heartRate, inplace=True)
heart_data['glucose'].fillna(mean_glucose, inplace=True)
heart_data['TenYearCHD'].fillna(mean_TenYearCHD, inplace=True)

# Print the count of missing values after filling
missing_values = heart_data.isnull().sum()
print(missing_values)

id                 0
age                0
education          0
sex                0
is_smoking         0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64


In [188]:
#ALTERNATIVELY
columns_to_fill = ['education', 'cigsPerDay', 'BPMeds', 'totChol', 'BMI', 'heartRate', 'glucose', 'TenYearCHD']

for column in columns_to_fill:
    mean_value = heart_data[column].mean()
    heart_data[column].fillna(mean_value, inplace=True)

missing_values = heart_data.isnull().sum()
print(missing_values)

id                 0
age                0
education          0
sex                0
is_smoking         0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64


In [189]:
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               4238 non-null   int64  
 1   age              4238 non-null   int64  
 2   education        4238 non-null   float64
 3   sex              4238 non-null   object 
 4   is_smoking       4238 non-null   object 
 5   cigsPerDay       4238 non-null   float64
 6   BPMeds           4238 non-null   float64
 7   prevalentStroke  4238 non-null   int64  
 8   prevalentHyp     4238 non-null   int64  
 9   diabetes         4238 non-null   int64  
 10  totChol          4238 non-null   float64
 11  sysBP            4238 non-null   float64
 12  diaBP            4238 non-null   float64
 13  BMI              4238 non-null   float64
 14  heartRate        4238 non-null   float64
 15  glucose          4238 non-null   float64
 16  TenYearCHD       4238 non-null   float64
dtypes: float64(10)

In [190]:
heart_data.head()

,id,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,0,64,2.0,F,YES,3.0,0.0,0,0,0,221.0,148.0,85.0,25.802008,90.0,80.0,1.0
1,1,36,4.0,M,NO,0.0,0.0,0,1,0,212.0,168.0,98.0,29.770000,72.0,75.0,0.0
2,2,46,1.0,F,YES,10.0,0.0,0,0,0,250.0,116.0,71.0,20.350000,88.0,94.0,0.0
3,3,50,1.0,M,YES,20.0,0.0,0,1,0,233.0,158.0,88.0,28.260000,68.0,94.0,1.0
4,4,64,1.0,F,YES,30.0,0.0,0,0,0,241.0,136.5,85.0,26.420000,70.0,77.0,0.0


In [191]:
X = heart_data.drop('TenYearCHD', axis=1)
y = heart_data['TenYearCHD']

In [192]:
# Convert categorical variables to numerical using one-hot encoding
X = pd.get_dummies(X, columns=['sex', 'is_smoking'])

In [193]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)

In [194]:
print(y_train.unique())

[0.         0.15073746 1.        ]


In [195]:
threshold = 0.5
y_train_binary = (y_train > threshold).astype(int)
print(y_train_binary.unique())

[0 1]


In [196]:
logistic_model = LogisticRegression(max_iter=5000)
logistic_model.fit(X_train, y_train_binary)

LogisticRegression(max_iter=5000)

In [198]:
y_pred = logistic_model.predict(X_test)
y_test_binary = (y_test > threshold).astype(int)

# Confusion matrix
conf_matrix = confusion_matrix(y_test_binary, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[743   3]
 [ 94   8]]


In [201]:
# ROC / AUC
fpr, tpr, thresholds = roc_curve(y_test_binary, logistic_model.predict_proba(X_test)[:, 1])
roc_auc = auc(fpr, tpr)

fig = go.Figure()

fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC curve (area = {roc_auc:.2f})'))
fig.add_shape(
    type='line',
    line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_layout(
    title='Receiver Operating Characteristic (ROC) Curve',
    xaxis=dict(title='False Positive Rate'),
    yaxis=dict(title='True Positive Rate'),
    legend=dict(x=0.01, y=0.99),
)

fig.show()

In [203]:
# AUC score
print("AUC Score:", roc_auc_score(y_test_binary, logistic_model.predict_proba(X_test)[:, 1]))

AUC Score: 0.7171844609157335


Explanation of ROC / AUC:
Receiver Operating Characteristic (ROC) Curve: It is a graphical representation of the model's ability to discriminate between positive and negative classes. It plots the true positive rate (sensitivity) against the false positive rate (1 - specificity) for different probability thresholds.

Area Under the Curve (AUC): It quantifies the overall performance of a classification model. AUC ranges from 0 to 1, where a higher AUC indicates better performance. AUC provides a single scalar value to compare different models. A model with an AUC of 0.5 is no better than random, while an AUC of 1.0 indicates a perfect classifier.

An AUC (Area Under the Curve) score of 0.717 suggests that this logistic regression model is performing moderately well in distinguishing between the two classes, however, there is still room for improvement.